In [6]:
import sys
import os
import glob

import Bio.SeqIO

import dms_variants.codonvarianttable
import alignparse.utils

import pandas as pd
pd.set_option('display.max_rows',10)

import seaborn as sns

import scipy as sp
import numpy as np

import matplotlib.pyplot as plt

In [7]:
bkg_dict = {'USSR77':'ATGAAAGCAAAACTACTGGTCCTGTTATGTGCACTTTCAGCTACAGATGCAGACACAATATGTATAGGCTACCATGCGAACAACTCAACCGACACTGTTGACACAGTACTCGAAAAGAACGTGACAGTGACACACTCTGTCAACCTACTTGAGGACAGTCACAACGGAAAACTATGCAGACTAAAAGGAATAGCCCCACTACAATTGGGGAAATGCAACATTGCCGGATGGATCTTAGGAAACCCAGAATGCGAATCACTGTTTTCTAAGAAATCATGGTCCTACATTGCAGAAACACCAAACTCCGAGAATGGAACATGTTACCCAGGATATTTCGCCGACTATGAGGAATTGAGGGAGCAATTGAGCTCAGTATCATCATTCGAGAGATTCGAAATATTCCCCAAGGAAAGATCATGGCCCAAACACAACGTAACCAGAGGCGTAACGGCATCATGCTCCCATAAGGGGAAAAGCAGTTTTTACAGAAATTTGCTATGGCTGACGGAGAAAAATGGCTCGTACCCAAATCTGAGCAAGTCCTATGTGAACAACAAAGAGAAAGAAGTCCTTGTACTATGGGGTGTTCATCACCCGTCAAACATAGAGGACCAAAAGACCATCTATCGGAAAGAAAATGCTTATGTGTCTGTAGTGTCTTCAAATTATAACAGGAGATTCACCCCAGAAATAGCAGAAAGACCCAAAGTAAGAGGTCAAGCAGGGAGAATTAACTACTACTGGACTCTGCTGGAACCCGGGGACACAATAATATTTGAGGCAAATGGAAATCTAATAGCGCCATGGCATGCTTTCGCACTGAATAGAGGCTTTGGGTCAGGAATCATCACCTCAAACGCATCGATGGATGAATGTGACACGAAGTGTCAAACACCCCAGGGAGCTATAAACAGTAGTCTTCCTTTCCAGAATATACACCCAGTCACAATAGGGGAGTGCCCAAAATACGTCAGGAGTACAAAATTGAGGATGGTTACAGGACTAAGGAACATCCCATCCATTCAATCCAGAGGTCTGTTTGGAGCCATTGCCGGTTTCATTGAAGGGGGATGGACTGGAATGATAGATGGATGGTATGGTTATCATCATCAGAATGAACAGGGATCTGGCTATGCTGCGGATCAAAAAAGCACACAAAATGCCATTAACGGGATTACAAACAAGGTGAACTCTGTTATCGAGAAAATGAACACTCAATTCACAGCTGTGGGTAAAGAATTCAACAAATTAGAAAAAAGGATGGAAAACTTAAATAAAAAAGTTGATGATGGATTTCTGGACATTTGGACATATAATGCAGAATTGTTGGTTCTACTGGAAAATGAAAGGACTTTGGATTTTCATGACTCAAATGTGAAGAATCTGTATGAGAAAGTAAAAAGCCAATTAAAGAATAATGCCAAAGAAATAGGAAACGGGTGTTTTGAATTCTACCACAAGTGTAACAATGAATGCATGGAAAGTGTAAAAAATGGAACTTATGACTATCCAAAATATTCAGAGGAATCAAAGTTAAACAGGGAAAAAATTGATGGAGTGAAATTGGAATCAATGGGAGTCTATCAGATTCTGGCGATCTACTCAACTGTCGCCAGTTCACTGGTCCTGCTCGTGTCACTGGGAGCCATCAGCTTCTGGATGTGCAGCAACGGCAGCCTCCAGTGTAGGATTTGCATTTAA',
            'SN89':'GCCCCACTACAATTGGGTAATTGCAGCATTGCCGGATGGATCTTAGGAAACCCAGAATGCGAATCACTGTTTTCTAAGGAATCATGGTCCTACATTGCAGAAACACCAAACTCCGAGAATGGAACATGTTACCCAGGGTATTTCGCCGACTATGAGGAACTGAGGGAGCAATTGAGTTCAGTATCATCATTCGAGAGATTCGAAATATTCCCCAAAGAAAGCTCATGGCCCAACCACACCGTAACCAAAGGAGTAACGGCAGCATGCTCCCATAATGGGAAAAGCAGTTTTTACAGAAATTTGCTATGGCTGACGGAGAAGAATGGCTTGTACCCAAATCTGAGCAAGTCCTATGTAAACAACAAAGAGAAAGAAGTCCTTGTACTATGGGGTGTTCATCACCCGTCAAACATAGGGGACCAAAGGGCCATCTATCATACAGAAAATGCTTATGTGTCTGTAGTGTCTTCACATTATAGCAGGAGATTCACCCCAGAAATAGCAAAAAGACCCAAAGTAAGAGGTCAAGAAGGAAGAATTAACTACTACTGGACTCTGCTGGAACCCGGGGACACAATAATATTTGAGGCAAATGGAAATCTAATAGCGCCATGGTATGCTTTCGCACTGAGTAGAGGCTTTGGGTCAGGAATCATCACCTCAAACGCATCAATGGATGAATGTGACGCGAAGTGTCAAACACCCCAGGGAGCTATAAACAGTAGTCTTCCTTTCCAGAATGTACACCCAGTCACAATAGGAGAGTGTCCAAAGTATGTCAGGAGTACAAAATTAAGGATGGTTACAGGACTAAGGAACATCCCATCCGTTCAATCCAGAGGTTTGTTTGGAGCCATTGCCGGTTTCATTGAAGGGGGGTGGACTGGAATGATAGATGGATGGTATGGTTATCATCATCAAAATGAACAAGGATCTGGCTATGCTGCGGATCAAAAAAGCACACAAAATGCCATTAACGGGATTACAAACAAGGTGAATTCTGTAATCGAGAAAATGAACACTCAATTCACAGCTGTGGGCAAAGAATTCAACAAATTAGAAAGAAGGATGGAAAACTTAAATAAAAAAGTTGATGATGGATTTCTGGACATTTGGACATATAATGCAGAATTGTTGGTTCTACTGGAAAATGAAAGGACTTTGGATTTTCATGACTCAAATGTGAAGAATCTGTATGAGAAAGTAAAAAGCCAATTAAAGAATAATGCCAAAGAAATAGGAAACGGGTGTTTTGAATTCTACCACAAGTGTAACAATGAATGCATGGAAAGTGTGAAAAATGGAACTTATGACTATCCAAAATATTCCGAGGAATCAAAGTTAAACAGGGAAAAAATTGATGGAGTGAAATTGGAATCAATGGGAGTCTATCAGATTCTGGCGATCTACTCAACTGTCGCCAGTTCACTGGTCCTGCTCGTGTCACTGGGAGCCATCAGCTTCTGGATGTGCAGCAACGGCAGCCTCCAGTGTAGGATTTGCATTTAA',
            'SI06':'ATGAAAGTAAAACTACTGGTTCTGTTATGCACATTTACAGCTACATATGCAGACACAATATGTATAGGCTACCATGCCAACAACTCAACCGACACTGTTGACACAGTACTTGAGAAGAATGTGACAGTGACACACTCTGTCAACCTGCTTGAGGACAGTCACAATGGAAAATTATGTCTATTAAAAGGAATAGCCCCACTACAATTGGGTAATTGCAGCGTTGCCGGATGGATCTTAGGAAACCCAGAATGCGAATTACTGATTTCCAGGGAATCATGGTCCTACATTGTAGAAAAACCAAATCCTGAGAATGGAACATGTTACCCAGGGCATTTCGCCGACTATGAGGAACTGAGGGAGCAATTGAGTTCAGTATCTTCATTTGAGAGATTCGAAATATTCCCCAAAGAAAGCTCATGGCCCAACCACACCACAACCGGAGTATCAGCATCATGCTCCCATAATGGGGAAAGCAGTTTTTACAAAAATTTGCTATGGCTGACGGGGAAGAATGGTTTGTACCCAAACCTGAGCAAGTCCTATGCAAACAACAAAGAGAAAGAAGTCCTTGTACTATGGGGTGTTCATCACCCGCCTAACATAGGTGACCAAAGGGCTCTCTATCATACAGAAAATGCTTATGTGTCTGTAGTGTCTTCACATTATAGCAGAAAATTCACCCCAGAAATAGCCAAAAGACCCAAAGTAAGAGATCAAGAAGGAAGAATCAACTACTACTGGACTCTACTTGAACCCGGGGATACAATAATATTTGAGGCAAATGGAAATCTAATAGCGCCAAGATATGCTTTCGCACTGAGTAGAGGCTTTGGATCAGGAATCATCAACTCAAATGCACCAATGGATGAATGTGATGCGAAGTGCCAAACACCTCAGGGAGCTATAAACAGCAGTCTTCCTTTCCAGAATGTACACCCTGTCACAATAGGAGAGTGTCCAAAGTATGTCAGGAGTGCAAAATTAAGGATGGTTACAGGACTAAGGAACATCCCATCCATTCAATCCAGAGGTTTGTTTGGAGCCATTGCCGGTTTCATTGAAGGGGGGTGGACTGGAATGGTAGATGGTTGGTATGGTTATCATCATCAGAATGAGCAAGGATCTGGCTATGCTGCAGATCAAAAAAGCACACAAAATGCCATTAATGGGATTACAAACAAGGTGAATTCTGTAATTGAGAAAATGAACACTCAATTCACAGCTGTGGGCAAAGAATTCAACAAATTGGAAAGAAGGATGGAAAACTTAAATAAAAAAGTTGATGATGGGTTTATAGACATTTGGACATATAATGCAGAATTGTTGGTTCTACTGGAAAATGAAAGGACTTTGGATTTCCATGACTCCAATGTGAAGAATCTGTATGAGAAAGTAAAAAGCCAATTAAAGAATAATGCCAAAGAAATAGGAAATGGGTGTTTTGAATTCTATCACAAGTGTAACGATGAATGCATGGAGAGTGTAAAAAATGGAACTTATGACTATCCAAAATATTCCGAAGAATCAAAGTTAAACAGGGAGAAAATTGATGGAGTGAAATTGGAATCAATGGGAGTCTATCAGATTCTGGCGATCTACTCAACAGTCGCCAGTTCTCTGGTCCTGCTCGTGTCACTGGGAGCCATCAGCTTCTGGATGTGCAGCAACGGCAGCCTCCAGTGTAGGATTTGCATTTAA',
            'SYD21':'ATGAAGGCAATACTAGTAGTTATGCTGTATACATTTACAACCGCAAATGCAGACACATTATGTATAGGTTATCATGCGAACAATTCAACAGACACTGTGGACACAGTACTAGAAAAGAATGTAACAGTAACACACTCTGTCAATCTTCTAGAAGACAAGCATAACGGAAAACTATGCAAACTAAGAGGGGTAGCCCCATTGCATTTGGGTCAATGTAACATTGCTGGCTGGATCCTGGGAAATCCAGAGTGTGAATCACTCTCCACAGCAAGATCATGGTCCTACATTGTGGAAACATCTAATTCAGACAATGGAACGTGTTACCCAGGAAATTTCATCAATTATGAGGAGCTAAGAGAGCAATTGAGCTCAGTGTCATCATTTGAAAGGTTTGAAATATTCCCCAAGACAAGTTCATGGCCTAATCATGACTCGGACAATGGTGTAACGGCAGCATGTCCTCACGCTGGAGCAAAAAGCTTCTACAAAAACTTGATATGGCTGGTTAAAAAAGGAAAATCATACCCAAAGATCAACCAAACCTACATTAATGATAAAGGGAAAGAAGTCCTCGTGCTGTGGGGCATTCACCATCCACCTACTATTACTGACCAAGAAAGTCTCTATCAGAATGCAGATGCATATGTTTTTGTGGGGACATCAAGATACAGCAAGAAGTTCAAGCCGGAAATAGCAGCAAGACCCAAAGTGAGGGATCAAGCAGGGAGAATGAACTATTACTGGACACTAGTAGAACCGGGAGATAAAATAACATTCGAAGCAACTGGTAATCTAGTGGCACCGAGATATGCATTCACAATGGAAAAAGATGCTGGATCTGGTATTATCATTTCAGATACACCAGTCCACGATTGCAATACAACTTGTCAGACACCCGAGGGTGCTATAAACACCAGCCTCCCATTTCAGAATGTACATCCGATCACGATTGGGAAATGTCCAAAGTATGTAAGAAGCACAAAATTGAGACTGGCCACAGGATTGAGGAATGTCCCGTCTATTCAATCTAGAGGCCTATTCGGGGCCATTGCTGGCTTCATCGAAGGGGGGTGGACAGGGATGGTAGATGGATGGTACGGTTATCACCATCAAAATGAGCAGGGGTCAGGATATGCAGCCGATCTGAAGAGCACACAAAATGCCATTGATAAGATTACCAACAAAGTAAATTCTGTCATTGAAAAGATGAATACACAGTTCACAGCAGTTGGTAAAGAGTTCAACCACCTTGAAAAAAGAATAGAGAATCTAAATAAAAAGGTTGATGATGGTTTCCTGGACATTTGGACTTACAATGCCGAACTGTTGGTTCTACTGGAAAACGAAAGAACTTTGGACTATCACGATTCAAATGTGAAGAACTTGTATGAAAAAGTAAGAAACCAGTTAAAAAACAATGCCAAGGAAATTGGAAACGGCTGCTTTGAATTTTACCACAAATGCGACAACACATGCATGGAAAGTGTCAAGAATGGGACTTATGACTACCCAAAATACTCAGAGGAAGCAAAATTAAACAGAGAAAAAATAGATGGAGTAAAGCTGGACTCAACAAGGATTTACCAGATTTTGGCAATCTATTCAACTGTTGCCAGTTCATTGGTCCTGGTCGTGTCACTGGGAGCCATCAGCTTCTGGATGTGCAGCAACGGCAGCCTCCAGTGTAGGATTTGCATTTAA'}

In [3]:
#unnecessary
variants_df = pd.DataFrame()

for bkg,seq in bkg_dict.items():
    variants = dms_variants.codonvarianttable.CodonVariantTable(
        barcode_variant_file=f'data/codon_variant_tables/codon_variant_table_{bkg}.csv',
        geneseq=seq,
        allowgaps=True,
        substitutions_are_codon=True,
        primary_target=bkg,
        substitutions_col="codon_substitutions",
    )
    
    temp_df = variants.barcode_variant_df
    variants_df = pd.concat([variants_df, temp_df], ignore_index=True)

variants_df

,target,library,barcode,variant_call_support,codon_substitutions,aa_substitutions,n_codon_substitutions,n_aa_substitutions
0,USSR77,rep1,ACAACTGTAAGTAGCTAGGCTGTGCGGC,3,GTA125CAA,V125Q,1,1
1,USSR77,rep1,ACAAGATCGGTCAGCTCAGTATCTGAAT,1,GTG216GAT,V216D,1,1
2,USSR77,rep1,ACAAGTGGAGAGAGCTCCAATTATCAAT,1,AAT275TCA,N275S,1,1
3,USSR77,rep1,ACAATAATAGCGAGCTAACCTCTGTCAA,3,TCA221TAT,S221Y,1,1
4,USSR77,rep1,ACAATTGCATGTAGCTGACGAAATTCCA,1,TTC112CCA,F112P,1,1
...,...,...,...,...,...,...,...,...
1043256,SYD21,rep2,TTGCGGTCCATTAGCTGCGATCGCCCCC,1,AGG130CAT,R130H,1,1
1043257,SYD21,rep2,TTGCGTACATCCAGCTCGGATGGTGCCT,2,AAT142GGA,N142G,1,1
1043258,SYD21,rep2,TTGCTAAGTACCAGCTCTACCAAAATGC,1,GGA110AAT,G110N,1,1
1043259,SYD21,rep2,TTGCTCACGCTCAGCTAGATATACAATC,5,GGA80TGC,G80C,1,1


In [8]:
# get all of the variant counts for each background, replicate, and selection
variant_counts_df = pd.DataFrame()

replicate_list = ['R1','R2']
selection_dict = {'USSR77':['NoAb','Ab6649','Ab9207','ADI-77474','ADI-77470'],
                 'SN89':['NoAb','UCA860','CH65','CH65_ND','UCA652','H2227','H1244','H1825','H679','UCA641','H1270','H1261','Ab6649','Ab9207','ADI-77474','ADI-77470'],
                 'SI06':['NoAb','CH65','H2227','H1244','Ab6649','ADI-77474','ADI-77470'],
                 'SYD21':['NoAb','ADI-77474','ADI-77470','ADI-86325']}

for bkg in selection_dict:
    for rep in replicate_list:
        for sel in selection_dict[bkg]:
            
            temp_df = pd.read_csv(f'data/counts_files/{bkg}_{rep}_{sel}.csv',keep_default_na=False)
            temp_df['background'] = bkg
            temp_df['replicate'] = rep
            temp_df['selection'] = sel
            
            variant_counts_df = pd.concat([variant_counts_df, temp_df], ignore_index=True)

variant_counts_df

,Unnamed: 0,barcode,count,codon_substitutions,aa_substitutions,variant_call_support,background,replicate,selection
0,21409,GCAAGTTAACGGAGCTAGATAGGTCTTT,58134,GGC277---,G277-,6,USSR77,R1,NoAb
1,6481,GCAAATGTCGGGAGCTATTAACTGCTTT,52882,GTA237GAA,V237E,6,USSR77,R1,NoAb
2,17222,GCAAGGATGAAAAGCTGGTAGTATCTCT,46384,GCA241GAA,A241E,4,USSR77,R1,NoAb
3,20220,GCAAGTACATCCAGCTTGACAGATAATT,35724,AAA142AAT,K142N,6,USSR77,R1,NoAb
4,1414,GCAAACAAACTGAGCTCTGATGAGCGGT,31955,GTT196CAT,V196H,6,USSR77,R1,NoAb
...,...,...,...,...,...,...,...,...,...
319917,1779,ATCTGGCTAAGAAGCTTGGCTAACTCCA,1,GCA212AAA,A212K,2,SYD21,R2,ADI-86325
319918,2331,ATCTTATCCAGTAGCTGTCAAAAGGATG,1,CCA66GGA,P66G,2,SYD21,R2,ADI-86325
319919,533,ATCTAGTTGTTGAGCTCAAATAGTGAGA,1,TCA102GAA,S102E,6,SYD21,R2,ADI-86325
319920,1545,ATCTGAGATTGTAGCTTATTCCTGGTCA,1,ATA166TTC,I166F,3,SYD21,R2,ADI-86325


In [9]:
# first aggregate the variant counts by aa_substitution rather than barcode
variants_agg = variant_counts_df.groupby(['background','replicate','selection','aa_substitutions']).agg({'count':'sum'}).reset_index()

# filter out variants that are not sequenced by > min_count
min_count = 10
variants_agg = variants_agg.query(f"count >= {min_count}")

# add a column that contains just the position
variants_agg['sequential_site'] = variants_agg['aa_substitutions'].str[1:-1]

# for each background, load the file that renumbers the positions to H3 numbering
renumber_df = pd.DataFrame()
for bkg in bkg_dict:
    temp_df=pd.read_csv(f'data/renumbering_files/{bkg}_renumber.csv',dtype=str, keep_default_na=False)
    temp_df['background'] = bkg
    temp_df = temp_df.drop(columns=['positional_barcode'])
    renumber_df = pd.concat([renumber_df,temp_df],ignore_index=True)

# get the wild-type data
wts = variants_agg.query("aa_substitutions == ''")

# renumber to H3 numbering
variants_agg = variants_agg.merge(renumber_df,on=['sequential_site','background'])

# add back in the wild-type data and fill the reference site with 'wt'
variants_agg = pd.concat([variants_agg,wts],ignore_index=True)
variants_agg['reference_site'] = variants_agg['reference_site'].fillna('wt')


variants_agg['aa_substitutions'] = variants_agg['aa_substitutions'].str[0:1]+variants_agg['reference_site']+variants_agg['aa_substitutions'].str[-1:]
variants_agg['mut_aa'] = variants_agg['aa_substitutions'].str[-1:]
variants_agg = variants_agg.query("mut_aa != '*' ")
variants_agg = variants_agg.drop(columns=['mut_aa'])

variants_agg

,background,replicate,selection,aa_substitutions,count,sequential_site,reference_site
0,SI06,R1,ADI-77470,A137L,1590,150,137
1,SI06,R1,ADI-77470,A137Q,282,150,137
2,SI06,R1,ADI-77470,A137V,12,150,137
3,SI06,R1,ADI-77474,A137D,1544,150,137
4,SI06,R1,ADI-77474,A137G,36,150,137
...,...,...,...,...,...,...,...
88562,USSR77,R2,ADI-77470,wt,541417,,wt
88563,USSR77,R2,ADI-77474,wt,197244,,wt
88564,USSR77,R2,Ab6649,wt,393176,,wt
88565,USSR77,R2,Ab9207,wt,349099,,wt


In [10]:
replicate_list = ['R1','R2']
selection_dict = {'USSR77':['Ab6649','Ab9207','ADI-77474','ADI-77470'],
                 'SN89':['UCA860','CH65','CH65_ND','UCA652','H2227','H1244','H1825','H679','UCA641','H1270','H1261','Ab6649','Ab9207','ADI-77474','ADI-77470'],
                 'SI06':['CH65','H2227','H1244','Ab6649','ADI-77474','ADI-77470'],
                 'SYD21':['ADI-77474','ADI-77470','ADI-86325']}

site_counts_df = variants_agg.groupby(['background','replicate','selection','reference_site']).agg({'count':'sum'}).reset_index()
P = 10

for bkg in selection_dict:
    for rep in replicate_list:
        for sel in selection_dict[bkg]:
            
            site_counts_df_sel = site_counts_df.query(f" background == '{bkg}' & replicate == '{rep}' & selection == '{sel}' ")
            site_counts_df_mock = site_counts_df.query(f" background == '{bkg}' & replicate == '{rep}' & selection == 'NoAb' ")
            site_counts_df_merged = site_counts_df_mock.merge(site_counts_df_sel, on = 'reference_site', suffixes=('_no_ab','_ab'))

            site_counts_df_merged['fsel'] = site_counts_df_merged['count_ab']/site_counts_df_merged['count_no_ab']
            site_counts_df_merged['fmock'] = site_counts_df_merged['count_no_ab']/site_counts_df_merged['count_ab']

            for idx,row in site_counts_df_merged.iterrows():
                if row['fsel']<1:
                    site_counts_df_merged.at[idx,'fsel']=1
                if row['fmock']<1:
                    site_counts_df_merged.at[idx,'fmock']=1   

            cols = ['reference_site','fsel','fmock']
            f_values = site_counts_df_merged[cols]


            sel_df = variants_agg.query(f" background == '{bkg}' & replicate == '{rep}' & selection == '{sel}' ")
            mock_df = variants_agg.query(f" background == '{bkg}' & replicate == '{rep}' & selection == 'NoAb' ")

            merged_df = mock_df.merge(sel_df, on = ['background','replicate','aa_substitutions','reference_site'], suffixes=('_no_ab', '_ab'))
            merged_df = merged_df.drop(columns=['selection_no_ab','sequential_site_no_ab','sequential_site_ab'])
            merged_df

            merged_df = merged_df.merge(f_values, on = 'reference_site')

            wt_sel = merged_df.query("aa_substitutions == 'wt' ").reset_index()['count_ab'].iat[0]
            wt_mock = merged_df.query("aa_substitutions == 'wt' ").reset_index()['count_no_ab'].iat[0]

            merged_df['diffsel'] = np.log2(((merged_df['count_ab']+merged_df['fsel']*P)/(wt_sel+merged_df['fsel']*P))/\
                                          ((merged_df['count_no_ab']+merged_df['fmock']*P)/(wt_mock+merged_df['fmock']*P)))

            merged_df = merged_df.sort_values(by='diffsel',ascending=False)

            merged_df.to_csv(f'data/diffsel_output/{bkg}_{rep}_{sel}_diffsel.csv')

In [125]:
#testing
replicate_list = ['R1','R2']
selection_dict = {'USSR77':['Ab6649','Ab9207','ADI-77474','ADI-77470'],
                 'SN89':['UCA860','CH65','CH65_ND','UCA652','H2227','H1244','H1825','H679','UCA641','H1270','H1261','Ab6649','Ab9207','ADI-77474','ADI-77470'],
                 'SI06':['CH65','H2227','H1244','Ab6649','ADI-77474','ADI-77470'],
                 'SYD21':['ADI-77474','ADI-77470','ADI-86325']}

P = 10
site_counts_df = variants_agg.groupby(['background','replicate','selection','reference_site']).agg({'count':'sum'}).reset_index()


for bkg in selection_dict:
    for rep in replicate_list:
        for sel in selection_dict[bkg]:
            
            site_counts_df_sel = site_counts_df.query(f" background == '{bkg}' & replicate == '{rep}' & selection == '{sel}' ")
            site_counts_df_mock = site_counts_df.query(f" background == '{bkg}' & replicate == '{rep}' & selection == 'NoAb' ")
            site_counts_df_merged = site_counts_df_mock.merge(site_counts_df_sel, on = 'reference_site', how='outer',suffixes=('_no_ab','_ab'))
            site_counts_df_merged['count_ab'] = site_counts_df_merged['count_ab'].fillna(site_counts_df_merged['count_no_ab'])
            site_counts_df_merged['count_no_ab'] = site_counts_df_merged['count_no_ab'].fillna(site_counts_df_merged['count_ab'])
            

            site_counts_df_merged['fsel'] = site_counts_df_merged['count_ab']/site_counts_df_merged['count_no_ab']
            site_counts_df_merged['fmock'] = site_counts_df_merged['count_no_ab']/site_counts_df_merged['count_ab']

            for idx,row in site_counts_df_merged.iterrows():
                if row['fsel']<1:
                    site_counts_df_merged.at[idx,'fsel']=1
                if row['fmock']<1:
                    site_counts_df_merged.at[idx,'fmock']=1   

            cols = ['reference_site','fsel','fmock']
            f_values = site_counts_df_merged[cols]


            sel_df = variants_agg.query(f" background == '{bkg}' & replicate == '{rep}' & selection == '{sel}' ")
            mock_df = variants_agg.query(f" background == '{bkg}' & replicate == '{rep}' & selection == 'NoAb' ")

            merged_df = mock_df.merge(sel_df, on = ['background','replicate','aa_substitutions','reference_site'],how='outer', suffixes=('_no_ab', '_ab'))
            merged_df = merged_df.drop(columns=['selection_no_ab','sequential_site_no_ab','sequential_site_ab'])

            merged_df = merged_df.merge(f_values, on = 'reference_site')

            wt_sel = merged_df.query("aa_substitutions == 'wt' ").reset_index()['count_ab'].iat[0]
            wt_mock = merged_df.query("aa_substitutions == 'wt' ").reset_index()['count_no_ab'].iat[0]
            
            merged_df['count_ab'] = merged_df['count_ab'].fillna(merged_df['count_no_ab'])
            merged_df['count_no_ab'] = merged_df['count_no_ab'].fillna(merged_df['count_ab'])
            merged_df['diffsel'] = np.log2(((merged_df['count_ab']+merged_df['fsel']*P)/(wt_sel+merged_df['fsel']*P))/\
                                          ((merged_df['count_no_ab']+merged_df['fmock']*P)/(wt_mock+merged_df['fmock']*P)))

            merged_df = merged_df.sort_values(by='diffsel',ascending=False)#.dropna()
            

            merged_df.to_csv(f'data/diffsel_output/test_{bkg}_{rep}_{sel}_diffsel.csv')

In [11]:
merged_df

,background,replicate,aa_substitutions,count_no_ab,reference_site,selection_ab,count_ab,fsel,fmock,diffsel
690,SYD21,R2,P122D,15417,122,ADI-86325,839534,27.804123,1.000000,7.730300
691,SYD21,R2,P122E,403,122,ADI-86325,22084,27.804123,1.000000,7.722529
1058,SYD21,R2,T124N,458,124,ADI-86325,23469,1.237883,1.000000,7.615330
941,SYD21,R2,S125E,13304,125,ADI-86325,492771,4.706963,1.000000,7.176151
688,SYD21,R2,P122-,5512,122,ADI-86325,199976,27.804123,1.000000,7.144260
...,...,...,...,...,...,...,...,...,...,...
401,SYD21,R2,K123E,7362,123,ADI-86325,18,1.516386,1.000000,-5.829864
714,SYD21,R2,P140M,5619,140,ADI-86325,15,1.000000,4.636778,-5.857513
904,SYD21,R2,S110N,5903,110,ADI-86325,16,1.000000,7.096574,-5.877385
1228,SYD21,R2,I243V,3249,243,ADI-86325,11,1.000000,217.809524,-6.041520


In [29]:
variants_agg.query("background == 'SN89' & aa_substitutions == 'A193F' & selection == 'H1270' ")

,background,replicate,selection,aa_substitutions,count,sequential_site,reference_site
16484,SN89,R1,H1270,A193F,1124,143,193
16714,SN89,R2,H1270,A193F,439,143,193
